# TNG300 Group Catalog : HDF5 to Parquet

## Import Basic Libraries

In [1]:
import numpy as np
import pandas as pd
import glob
import sys
import h5py
#from netCDF4 import Dataset
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

import pyarrow as pa
import pyarrow.parquet as pq

from functools import reduce
import operator
import gc

In [2]:
# plot settings
plt.rc('font', family='serif') 
#plt.rc('font', serif='Times New Roman') 
plt.rcParams.update({'font.size': 16})
plt.rcParams['mathtext.fontset'] = 'stix'

## Define SparkSession and sparkContext

In [3]:
# PySpark packages
from pyspark import SparkContext   
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import Row
from pyspark.sql.window import Window as W


spark = SparkSession.builder \
    .master("yarn") \
    .appName("spark-shell") \
    .config("spark.driver.maxResultSize", "32g") \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "200") \
    .getOrCreate()


sc = spark.sparkContext
sc.setCheckpointDir("hdfs://spark00:54310/tmp/checkpoints")

spark.conf.set("spark.sql.debug.maxToStringFields", 500)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [4]:
sc.getConf().getAll()[:10]

[('spark.driver.memory', '32g'),
 ('spark.driver.appUIAddress', 'http://spark00:4040'),
 ('spark.ui.proxyBase', '/proxy/application_1742867806927_0005'),
 ('spark.app.id', 'application_1742867806927_0005'),
 ('spark.driver.maxResultSize', '32g'),
 ('spark.app.submitTime', '1749790492276'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://spark18:8088/proxy/application_1742867806927_0005'),
 ('spark.master', 'yarn'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client')]

## Read h5 files 

> Examples to handle group catalog, merger tree and snapshot data can be found at  https://www.tng-project.org/data/docs/scripts/

In [5]:
h5dir = '/mnt/data/shong/tempdata/tng50snap84/raw/'

In [8]:
h5list = sorted(glob.glob(h5dir+'snap*.hdf5'))

In [9]:
h5list[:3]

['/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.0.hdf5',
 '/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.1.hdf5',
 '/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.10.hdf5']

In [10]:
numh5list = len(h5list)
print(numh5list)

680


In [11]:
h5py.is_hdf5(h5list[0])

True

In [12]:
inh5file = h5list[0]

In [13]:
try:
    h5f = h5py.File(inh5file, "r")
except IOError as e:
    print("Error opening HDF5 file:", str(e))
# Don't forget f.close() when done! 

## Explore the `h5`

In [14]:
h5f.keys()

<KeysViewHDF5 ['Config', 'Header', 'Parameters', 'PartType0', 'PartType1', 'PartType3', 'PartType4', 'PartType5']>

In [15]:
for key in h5f.keys():
    item = h5f[key]
    
    if isinstance(item, h5py.Group):  # Check if it's a group
        print(f"Group: {key}")
    elif isinstance(item, h5py.Dataset):  # Check if it's a dataset
        print(f"Dataset: {key}, dtype: {item.dtype}")
    else:
        print(f"Unknown item: {key}")

Group: Config
Group: Header
Group: Parameters
Group: PartType0
Group: PartType1
Group: PartType3
Group: PartType4
Group: PartType5


In [16]:
h5f['Config'].keys()

<KeysViewHDF5 []>

In [17]:
h5f['Header'].keys()

<KeysViewHDF5 []>

In [18]:
h5f['Parameters'].keys()

<KeysViewHDF5 []>

In [19]:
h5f['PartType0'].keys()

<KeysViewHDF5 ['CenterOfMass', 'Coordinates', 'Density', 'ElectronAbundance', 'EnergyDissipation', 'GFM_AGNRadiation', 'GFM_CoolingRate', 'GFM_Metallicity', 'GFM_Metals', 'GFM_MetalsTagged', 'GFM_WindDMVelDisp', 'GFM_WindHostHaloMass', 'InternalEnergy', 'InternalEnergyOld', 'Machnumber', 'MagneticField', 'MagneticFieldDivergence', 'Masses', 'NeutralHydrogenAbundance', 'ParticleIDs', 'Potential', 'StarFormationRate', 'SubfindDMDensity', 'SubfindDensity', 'SubfindHsml', 'SubfindVelDisp', 'Velocities']>

In [20]:
h5f['PartType1'].keys()

<KeysViewHDF5 ['Coordinates', 'ParticleIDs', 'Potential', 'SubfindDMDensity', 'SubfindDensity', 'SubfindHsml', 'SubfindVelDisp', 'Velocities']>

In [21]:
h5f['PartType3'].keys()

<KeysViewHDF5 ['ParentID', 'TracerID']>

In [22]:
h5f['PartType4'].keys()

<KeysViewHDF5 ['BirthPos', 'BirthVel', 'Coordinates', 'GFM_InitialMass', 'GFM_Metallicity', 'GFM_Metals', 'GFM_MetalsTagged', 'GFM_StellarFormationTime', 'GFM_StellarPhotometrics', 'Masses', 'ParticleIDs', 'Potential', 'StellarHsml', 'SubfindDMDensity', 'SubfindDensity', 'SubfindHsml', 'SubfindVelDisp', 'Velocities']>

In [23]:
h5f['PartType5'].keys()

<KeysViewHDF5 ['BH_BPressure', 'BH_CumEgyInjection_QM', 'BH_CumEgyInjection_RM', 'BH_CumMassGrowth_QM', 'BH_CumMassGrowth_RM', 'BH_Density', 'BH_HostHaloMass', 'BH_Hsml', 'BH_Mass', 'BH_Mdot', 'BH_MdotBondi', 'BH_MdotEddington', 'BH_Pressure', 'BH_Progs', 'BH_U', 'Coordinates', 'Masses', 'ParticleIDs', 'Potential', 'SubfindDMDensity', 'SubfindDensity', 'SubfindHsml', 'SubfindVelDisp', 'Velocities']>

#### My Guesses What each Particle Type Means 

- PartType0 : Gas?? Particle
- PartType1 : DM? Particle
- PartType3 : Dummy
- PartType4 : Star? Particle
- PartType5 : Black Hole Particle

## Target Hadoop Paths

In [29]:
!hdfs dfs -ls /common/data/illustris/tng50/snapshot84/

Found 4 items
drwxr-xr-x   - shong supergroup          0 2025-06-13 14:14 /common/data/illustris/tng50/snapshot84/bh
drwxr-xr-x   - shong supergroup          0 2025-06-13 14:14 /common/data/illustris/tng50/snapshot84/dm
drwxr-xr-x   - shong supergroup          0 2025-06-13 14:14 /common/data/illustris/tng50/snapshot84/gas
drwxr-xr-x   - shong supergroup          0 2025-06-13 14:14 /common/data/illustris/tng50/snapshot84/star


```
/common/data/illustris/tng50/snapshot84/gas
/common/data/illustris/tng50/snapshot84/dm
/common/data/illustris/tng50/snapshot84/star
/common/data/illustris/tng50/snapshot84/bh
```

In [30]:
hdfsgaspath = '/common/data/illustris/tng50/snapshot84/gas/'
hdfsdmpath = '/common/data/illustris/tng50/snapshot84/dm/'
hdfsstarpath = '/common/data/illustris/tng50/snapshot84/star/'
hdfsbhpath = '/common/data/illustris/tng50/snapshot84/bh/'

# Explore each `Group`

## `Gas` Particles

In [31]:
for key in h5f['PartType0'].keys():
    item = h5f['PartType0'][key]
    
    if isinstance(item, h5py.Group):  # Check if it's a group
        print(f"Group: {key}")
    elif isinstance(item, h5py.Dataset):  # Check if it's a dataset
        print(f"Dataset: {key}, dtype: {item.dtype}, shape: {item.shape}")
    else:
        print(f"Unknown item: {key}")

Dataset: CenterOfMass, dtype: float32, shape: (12740556, 3)
Dataset: Coordinates, dtype: float64, shape: (12740556, 3)
Dataset: Density, dtype: float32, shape: (12740556,)
Dataset: ElectronAbundance, dtype: float32, shape: (12740556,)
Dataset: EnergyDissipation, dtype: float32, shape: (12740556,)
Dataset: GFM_AGNRadiation, dtype: float32, shape: (12740556,)
Dataset: GFM_CoolingRate, dtype: float32, shape: (12740556,)
Dataset: GFM_Metallicity, dtype: float32, shape: (12740556,)
Dataset: GFM_Metals, dtype: float32, shape: (12740556, 10)
Dataset: GFM_MetalsTagged, dtype: float32, shape: (12740556, 6)
Dataset: GFM_WindDMVelDisp, dtype: float32, shape: (12740556,)
Dataset: GFM_WindHostHaloMass, dtype: float32, shape: (12740556,)
Dataset: InternalEnergy, dtype: float32, shape: (12740556,)
Dataset: InternalEnergyOld, dtype: float32, shape: (12740556,)
Dataset: Machnumber, dtype: float32, shape: (12740556,)
Dataset: MagneticField, dtype: float32, shape: (12740556, 3)
Dataset: MagneticFieldDive

In [36]:
h5f['PartType0/Density'][:4]

array([0.0014777 , 0.00014894, 0.00011977, 0.00052269], dtype=float32)

In [37]:
h5f['PartType0/GFM_MetalsTagged'][:4]

array([[0.00341575, 0.02955053, 0.03039325, 0.0113246 , 0.0018249 ,
        0.00107294],
       [0.00528119, 0.00395109, 0.06079636, 0.01422283, 0.00287897,
        0.00014479],
       [0.00523572, 0.00463134, 0.0593179 , 0.01603837, 0.00286632,
        0.00017363],
       [0.00412535, 0.01757455, 0.04118681, 0.01462806, 0.00224552,
        0.00062448]], dtype=float32)

In [38]:
h5f['PartType0/Coordinates'][:4]

array([[ 7312.49536211, 24564.95299761, 21472.6966363 ],
       [ 7311.74511646, 24565.17284081, 21472.77431082],
       [ 7311.36912136, 24565.32323254, 21472.7139865 ],
       [ 7311.85786465, 24564.66993331, 21472.51989424]])

#### Parquet Format

In [39]:
gas_schema = T.StructType([
    T.StructField("CenterOfMass", T.ArrayType(T.FloatType()), True),
    T.StructField("Coordinates", T.ArrayType(T.DoubleType()), True),
    T.StructField("Density", T.FloatType(), True),
    T.StructField("ElectronAbundance", T.FloatType(), True),
    T.StructField("EnergyDissipation", T.FloatType(), True),
    T.StructField("GFM_AGNRadiation", T.FloatType(), True),
    T.StructField("GFM_CoolingRate", T.FloatType(), True),
    T.StructField("GFM_Metallicity", T.FloatType(), True),
    T.StructField("GFM_Metals", T.ArrayType(T.FloatType()), True),
    T.StructField("GFM_MetalsTagged", T.ArrayType(T.FloatType()), True),
    T.StructField("GFM_WindDMVelDisp", T.FloatType(), True),
    T.StructField("GFM_WindHostHaloMass", T.FloatType(), True),
    T.StructField("InternalEnergy", T.FloatType(), True),
    T.StructField("InternalEnergyOld", T.FloatType(), True),
    T.StructField("Machnumber", T.FloatType(), True),
    T.StructField("MagneticField", T.ArrayType(T.FloatType()), True),
    T.StructField("MagneticFieldDivergence", T.FloatType(), True),
    T.StructField("Masses", T.FloatType(), True),
    T.StructField("NeutralHydrogenAbundance", T.FloatType(), True),
    T.StructField("ParticleIDs", T.LongType(), True),
    T.StructField("Potential", T.FloatType(), True),
    T.StructField("StarFormationRate", T.FloatType(), True),
    T.StructField("SubfindDMDensity", T.FloatType(), True),
    T.StructField("SubfindDensity", T.FloatType(), True),
    T.StructField("SubfindHsml", T.FloatType(), True),
    T.StructField("SubfindVelDisp", T.FloatType(), True),
    T.StructField("Velocities", T.ArrayType(T.FloatType()), True),
])

In [42]:
gas_group = h5f['PartType0']

In [43]:
%%time
# Convert datasets to dictionary of NumPy arrays: this causes some type errors. 
#Let's use `list`, not `np.array`
# Extract and convert each dataset to a list
gas_data = {
    key: gas_group[key][:].tolist()
    for key in gas_group.keys()
}

CPU times: user 45.7 s, sys: 8.14 s, total: 53.9 s
Wall time: 53.8 s


### EDA Code Blocks for `gas_data`

- from `gas_data`, we can directly make and save the final parquet file to trigger lazy evaluation as all-in-one
- the below is only for EDA testing purpose, with sluggish performances

In [44]:
# Optional: Confirm structure
for key, value in gas_data.items():
    print(f"{key}: type={type(value)}, shape={len(value)}")

CenterOfMass: type=<class 'list'>, shape=12740556
Coordinates: type=<class 'list'>, shape=12740556
Density: type=<class 'list'>, shape=12740556
ElectronAbundance: type=<class 'list'>, shape=12740556
EnergyDissipation: type=<class 'list'>, shape=12740556
GFM_AGNRadiation: type=<class 'list'>, shape=12740556
GFM_CoolingRate: type=<class 'list'>, shape=12740556
GFM_Metallicity: type=<class 'list'>, shape=12740556
GFM_Metals: type=<class 'list'>, shape=12740556
GFM_MetalsTagged: type=<class 'list'>, shape=12740556
GFM_WindDMVelDisp: type=<class 'list'>, shape=12740556
GFM_WindHostHaloMass: type=<class 'list'>, shape=12740556
InternalEnergy: type=<class 'list'>, shape=12740556
InternalEnergyOld: type=<class 'list'>, shape=12740556
Machnumber: type=<class 'list'>, shape=12740556
MagneticField: type=<class 'list'>, shape=12740556
MagneticFieldDivergence: type=<class 'list'>, shape=12740556
Masses: type=<class 'list'>, shape=12740556
NeutralHydrogenAbundance: type=<class 'list'>, shape=1274055

In [46]:
%%time
# Turn the dictionary into row-wise tuples
data_tuples = list(zip(*gas_data.values()))

CPU times: user 21.4 s, sys: 1.2 s, total: 22.6 s
Wall time: 22.6 s


In [48]:
%%time
gasdf = spark.createDataFrame(data_tuples, schema=gas_schema)

CPU times: user 3min 18s, sys: 2.73 s, total: 3min 21s
Wall time: 3min 23s


In [49]:
gasdf.printSchema()

root
 |-- CenterOfMass: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Coordinates: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- Density: float (nullable = true)
 |-- ElectronAbundance: float (nullable = true)
 |-- EnergyDissipation: float (nullable = true)
 |-- GFM_AGNRadiation: float (nullable = true)
 |-- GFM_CoolingRate: float (nullable = true)
 |-- GFM_Metallicity: float (nullable = true)
 |-- GFM_Metals: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- GFM_MetalsTagged: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- GFM_WindDMVelDisp: float (nullable = true)
 |-- GFM_WindHostHaloMass: float (nullable = true)
 |-- InternalEnergy: float (nullable = true)
 |-- InternalEnergyOld: float (nullable = true)
 |-- Machnumber: float (nullable = true)
 |-- MagneticField: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- MagneticFieldDivergence: float 

In [51]:
cols = gasdf.columns

In [56]:
len(cols)

27

#### Show some rows

> instead of `gasdf.show(5,truncate=True)`, do some works to display the row values better

In [65]:
%%time
rows = gasdf.take(5)  # Returns list of Row objects
rows_dict = [row.asDict() for row in rows]
df = pd.DataFrame(rows_dict)

# Transpose and label
df_transposed = df.transpose()
df_transposed.columns = [f"Row_{i}" for i in range(len(rows))]
print(df_transposed)

                                                                      Row_0  \
CenterOfMass               [7312.419921875, 24565.0078125, 21472.666015625]   
Coordinates               [7312.495362107541, 24564.95299761482, 21472.6...   
Density                                                            0.001478   
ElectronAbundance                                                  1.233613   
EnergyDissipation                                                       0.0   
GFM_AGNRadiation                                                   0.000028   
GFM_CoolingRate                                                        -0.0   
GFM_Metallicity                                                     0.06336   
GFM_Metals                [0.6383745670318604, 0.29826587438583374, 0.00...   
GFM_MetalsTagged          [0.0034157498739659786, 0.02955053374171257, 0...   
GFM_WindDMVelDisp                                               1201.722412   
GFM_WindHostHaloMass                                

### All-in-one Code Block for `gas_data`

#### Hadoop Path

In [68]:
hdfsheader = 'hdfs://spark00:54310'

In [69]:
hdfsheader + hdfsgaspath

'hdfs://spark00:54310/common/data/illustris/tng50/snapshot84/gas/'

#### Parsing File Name

In [70]:
inh5file

'/mnt/data/shong/tempdata/tng50snap84/raw/snap_084.0.hdf5'

In [73]:
parsedname = inh5file.split("/")[-1].replace(".hdf5", "")
print(parsedname)

snap_084.0


In [74]:
nametail = '_gas.parquet.snappy'

In [76]:
outgasname = hdfsheader + hdfsgaspath + parsedname + nametail
print(outgasname)

hdfs://spark00:54310/common/data/illustris/tng50/snapshot84/gas/snap_084.0_gas.parquet.snappy


#### Save with overwrite

In [77]:
%%time
gasdf.write.option("compression", "snappy") \
    .mode("overwrite") \
    .save(outgasname)

CPU times: user 21.9 ms, sys: 52 µs, total: 21.9 ms
Wall time: 1min 23s


## `DM` Particles

In [78]:
for key in h5f['PartType1'].keys():
    item = h5f['PartType1'][key]
    
    if isinstance(item, h5py.Group):  # Check if it's a group
        print(f"Group: {key}")
    elif isinstance(item, h5py.Dataset):  # Check if it's a dataset
        print(f"Dataset: {key}, dtype: {item.dtype}, shape: {item.shape}")
    else:
        print(f"Unknown item: {key}")

Dataset: Coordinates, dtype: float64, shape: (15072481, 3)
Dataset: ParticleIDs, dtype: uint64, shape: (15072481,)
Dataset: Potential, dtype: float32, shape: (15072481,)
Dataset: SubfindDMDensity, dtype: float32, shape: (15072481,)
Dataset: SubfindDensity, dtype: float32, shape: (15072481,)
Dataset: SubfindHsml, dtype: float32, shape: (15072481,)
Dataset: SubfindVelDisp, dtype: float32, shape: (15072481,)
Dataset: Velocities, dtype: float32, shape: (15072481, 3)


In [79]:
dm_schema = T.StructType([
    T.StructField("Coordinates", T.ArrayType(T.DoubleType()), True),
    T.StructField("ParticleIDs", T.LongType(), True),
    T.StructField("Potential", T.FloatType(), True),
    T.StructField("SubfindDMDensity", T.FloatType(), True),
    T.StructField("SubfindDensity", T.FloatType(), True),
    T.StructField("SubfindHsml", T.FloatType(), True),
    T.StructField("SubfindVelDisp", T.FloatType(), True),
    T.StructField("Velocities", T.ArrayType(T.FloatType()), True),
])

In [80]:
dm_group = h5f['PartType1']

In [81]:
%%time
# Convert datasets to dictionary of NumPy arrays: this causes some type errors. 
#Let's use `list`, not `np.array`
# Extract and convert each dataset to a list
dm_data = {
    key: dm_group[key][:].tolist()
    for key in dm_group.keys()
}

CPU times: user 16.7 s, sys: 1.97 s, total: 18.7 s
Wall time: 23.4 s


### EDA Code Blocks for `dm_data`

In [82]:
# Optional: Confirm structure
for key, value in dm_data.items():
    print(f"{key}: type={type(value)}, shape={len(value)}")

Coordinates: type=<class 'list'>, shape=15072481
ParticleIDs: type=<class 'list'>, shape=15072481
Potential: type=<class 'list'>, shape=15072481
SubfindDMDensity: type=<class 'list'>, shape=15072481
SubfindDensity: type=<class 'list'>, shape=15072481
SubfindHsml: type=<class 'list'>, shape=15072481
SubfindVelDisp: type=<class 'list'>, shape=15072481
Velocities: type=<class 'list'>, shape=15072481


In [83]:
%%time
# Turn the dictionary into row-wise tuples
dm_data_tuples = list(zip(*dm_data.values()))

CPU times: user 16.9 s, sys: 459 ms, total: 17.4 s
Wall time: 17.3 s


In [84]:
%%time
dmdf = spark.createDataFrame(dm_data_tuples, schema=dm_schema)

CPU times: user 1min 3s, sys: 808 ms, total: 1min 4s
Wall time: 1min 4s


In [85]:
dmdf.printSchema()

root
 |-- Coordinates: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- ParticleIDs: long (nullable = true)
 |-- Potential: float (nullable = true)
 |-- SubfindDMDensity: float (nullable = true)
 |-- SubfindDensity: float (nullable = true)
 |-- SubfindHsml: float (nullable = true)
 |-- SubfindVelDisp: float (nullable = true)
 |-- Velocities: array (nullable = true)
 |    |-- element: float (containsNull = true)



#### Show some rows

In [86]:
%%time
rows = dmdf.take(5)  # Returns list of Row objects
rows_dict = [row.asDict() for row in rows]
df = pd.DataFrame(rows_dict)

# Transpose and label
df_transposed = df.transpose()
df_transposed.columns = [f"Row_{i}" for i in range(len(rows))]
print(df_transposed)

                                                              Row_0  \
Coordinates       [7311.789333827647, 24565.490091139818, 21473....   
ParticleIDs                                              1700917146   
Potential                                                -4935796.0   
SubfindDMDensity                                           0.251378   
SubfindDensity                                            24.844238   
SubfindHsml                                                0.141662   
SubfindVelDisp                                          1535.332275   
Velocities        [141.73175048828125, 611.8182373046875, -339.2...   

                                                              Row_1  \
Coordinates       [7311.830562580984, 24565.235081401832, 21472....   
ParticleIDs                                              1454880154   
Potential                                                -4842780.0   
SubfindDMDensity                                           0.157524   
Subfi

### All-in-one Code Block for `dm_data`

In [87]:
hdfsheader + hdfsdmpath

'hdfs://spark00:54310/common/data/illustris/tng50/snapshot84/dm/'

In [88]:
dmnametail = '_dm.parquet.snappy'

In [89]:
outdmname = hdfsheader + hdfsdmpath + parsedname + dmnametail
print(outdmname)

hdfs://spark00:54310/common/data/illustris/tng50/snapshot84/dm/snap_084.0_dm.parquet.snappy


In [90]:
%%time
dmdf.write.option("compression", "snappy") \
    .mode("overwrite") \
    .save(outdmname)

CPU times: user 7.76 ms, sys: 7.99 ms, total: 15.7 ms
Wall time: 33 s


## `Star` Particle

In [91]:
for key in h5f['PartType4'].keys():
    item = h5f['PartType4'][key]
    
    if isinstance(item, h5py.Group):  # Check if it's a group
        print(f"Group: {key}")
    elif isinstance(item, h5py.Dataset):  # Check if it's a dataset
        print(f"Dataset: {key}, dtype: {item.dtype}, shape: {item.shape}")
    else:
        print(f"Unknown item: {key}")

Dataset: BirthPos, dtype: float32, shape: (1865063, 3)
Dataset: BirthVel, dtype: float32, shape: (1865063, 3)
Dataset: Coordinates, dtype: float64, shape: (1865063, 3)
Dataset: GFM_InitialMass, dtype: float32, shape: (1865063,)
Dataset: GFM_Metallicity, dtype: float32, shape: (1865063,)
Dataset: GFM_Metals, dtype: float32, shape: (1865063, 10)
Dataset: GFM_MetalsTagged, dtype: float32, shape: (1865063, 6)
Dataset: GFM_StellarFormationTime, dtype: float32, shape: (1865063,)
Dataset: GFM_StellarPhotometrics, dtype: float32, shape: (1865063, 8)
Dataset: Masses, dtype: float32, shape: (1865063,)
Dataset: ParticleIDs, dtype: uint64, shape: (1865063,)
Dataset: Potential, dtype: float32, shape: (1865063,)
Dataset: StellarHsml, dtype: float32, shape: (1865063,)
Dataset: SubfindDMDensity, dtype: float32, shape: (1865063,)
Dataset: SubfindDensity, dtype: float32, shape: (1865063,)
Dataset: SubfindHsml, dtype: float32, shape: (1865063,)
Dataset: SubfindVelDisp, dtype: float32, shape: (1865063,)
D

In [92]:
star_schema = T.StructType([
    T.StructField("BirthPos", T.ArrayType(T.FloatType()), True),
    T.StructField("BirthVel", T.ArrayType(T.FloatType()), True),
    T.StructField("Coordinates", T.ArrayType(T.DoubleType()), True),
    T.StructField("GFM_InitialMass", T.FloatType(), True),
    T.StructField("GFM_Metallicity", T.FloatType(), True),
    T.StructField("GFM_Metals", T.ArrayType(T.FloatType()), True),
    T.StructField("GFM_MetalsTagged", T.ArrayType(T.FloatType()), True),
    T.StructField("GFM_StellarFormationTime", T.FloatType(), True),
    T.StructField("GFM_StellarPhotometrics", T.ArrayType(T.FloatType()), True),
    T.StructField("Masses", T.FloatType(), True),
    T.StructField("ParticleIDs", T.LongType(), True),
    T.StructField("Potential", T.FloatType(), True),
    T.StructField("StellarHsml", T.FloatType(), True),
    T.StructField("SubfindDMDensity", T.FloatType(), True),
    T.StructField("SubfindDensity", T.FloatType(), True),
    T.StructField("SubfindHsml", T.FloatType(), True),
    T.StructField("SubfindVelDisp", T.FloatType(), True),
    T.StructField("Velocities", T.ArrayType(T.FloatType()), True),
])

In [93]:
star_group = h5f['PartType4']

In [94]:
%%time
# Convert datasets to dictionary of NumPy arrays: this causes some type errors. 
#Let's use `list`, not `np.array`
# Extract and convert each dataset to a list
star_data = {
    key: star_group[key][:].tolist()
    for key in star_group.keys()
}

CPU times: user 1.64 s, sys: 879 ms, total: 2.52 s
Wall time: 4.35 s


### EDA Code Blocks for `star_data`

In [95]:
# Optional: Confirm structure
for key, value in star_data.items():
    print(f"{key}: type={type(value)}, shape={len(value)}")

BirthPos: type=<class 'list'>, shape=1865063
BirthVel: type=<class 'list'>, shape=1865063
Coordinates: type=<class 'list'>, shape=1865063
GFM_InitialMass: type=<class 'list'>, shape=1865063
GFM_Metallicity: type=<class 'list'>, shape=1865063
GFM_Metals: type=<class 'list'>, shape=1865063
GFM_MetalsTagged: type=<class 'list'>, shape=1865063
GFM_StellarFormationTime: type=<class 'list'>, shape=1865063
GFM_StellarPhotometrics: type=<class 'list'>, shape=1865063
Masses: type=<class 'list'>, shape=1865063
ParticleIDs: type=<class 'list'>, shape=1865063
Potential: type=<class 'list'>, shape=1865063
StellarHsml: type=<class 'list'>, shape=1865063
SubfindDMDensity: type=<class 'list'>, shape=1865063
SubfindDensity: type=<class 'list'>, shape=1865063
SubfindHsml: type=<class 'list'>, shape=1865063
SubfindVelDisp: type=<class 'list'>, shape=1865063
Velocities: type=<class 'list'>, shape=1865063


In [96]:
%%time
# Turn the dictionary into row-wise tuples
star_data_tuples = list(zip(*star_data.values()))

CPU times: user 749 ms, sys: 100 ms, total: 849 ms
Wall time: 848 ms


In [97]:
%%time
stardf = spark.createDataFrame(star_data_tuples, schema=star_schema)

CPU times: user 27.9 s, sys: 356 ms, total: 28.3 s
Wall time: 28.4 s


In [98]:
stardf.printSchema()

root
 |-- BirthPos: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- BirthVel: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Coordinates: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- GFM_InitialMass: float (nullable = true)
 |-- GFM_Metallicity: float (nullable = true)
 |-- GFM_Metals: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- GFM_MetalsTagged: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- GFM_StellarFormationTime: float (nullable = true)
 |-- GFM_StellarPhotometrics: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Masses: float (nullable = true)
 |-- ParticleIDs: long (nullable = true)
 |-- Potential: float (nullable = true)
 |-- StellarHsml: float (nullable = true)
 |-- SubfindDMDensity: float (nullable = true)
 |-- SubfindDensity: float (nullable = true)
 |-- SubfindHsml: float (nullable = true)
 |-- Subf

In [99]:
%%time
rows = stardf.take(5)  # Returns list of Row objects
rows_dict = [row.asDict() for row in rows]
df = pd.DataFrame(rows_dict)

# Transpose and label
df_transposed = df.transpose()
df_transposed.columns = [f"Row_{i}" for i in range(len(rows))]
print(df_transposed)

                                                                      Row_0  \
BirthPos                    [7212.537109375, 24320.583984375, 21248.703125]   
BirthVel                  [0.8936818838119507, 535.03564453125, 459.4967...   
Coordinates               [7311.795642166191, 24565.493451227343, 21472....   
GFM_InitialMass                                                    0.000007   
GFM_Metallicity                                                    0.031656   
GFM_Metals                [0.697553813457489, 0.27079033851623535, 0.003...   
GFM_MetalsTagged          [0.001445992267690599, 0.023036668077111244, 0...   
GFM_StellarFormationTime                                           0.732088   
GFM_StellarPhotometrics   [-6.187445163726807, -6.531508445739746, -7.29...   
Masses                                                             0.000005   
ParticleIDs                                                    147599195192   
Potential                                           

### All-in-one Code Block for `star_data`

In [100]:
hdfsheader + hdfsstarpath

'hdfs://spark00:54310/common/data/illustris/tng50/snapshot84/star/'

In [101]:
starnametail = '_star.parquet.snappy'

In [103]:
outstarname = hdfsheader + hdfsstarpath + parsedname + starnametail
print(outstarname)

hdfs://spark00:54310/common/data/illustris/tng50/snapshot84/star/snap_084.0_star.parquet.snappy


In [104]:
%%time
stardf.write.option("compression", "snappy") \
    .mode("overwrite") \
    .save(outstarname)

CPU times: user 7.05 ms, sys: 4.15 ms, total: 11.2 ms
Wall time: 18.2 s


## `BH` Particles

In [105]:
for key in h5f['PartType5'].keys():
    item = h5f['PartType5'][key]
    
    if isinstance(item, h5py.Group):  # Check if it's a group
        print(f"Group: {key}")
    elif isinstance(item, h5py.Dataset):  # Check if it's a dataset
        print(f"Dataset: {key}, dtype: {item.dtype}, shape: {item.shape}")
    else:
        print(f"Unknown item: {key}")

Dataset: BH_BPressure, dtype: float32, shape: (8,)
Dataset: BH_CumEgyInjection_QM, dtype: float32, shape: (8,)
Dataset: BH_CumEgyInjection_RM, dtype: float32, shape: (8,)
Dataset: BH_CumMassGrowth_QM, dtype: float32, shape: (8,)
Dataset: BH_CumMassGrowth_RM, dtype: float32, shape: (8,)
Dataset: BH_Density, dtype: float32, shape: (8,)
Dataset: BH_HostHaloMass, dtype: float32, shape: (8,)
Dataset: BH_Hsml, dtype: float32, shape: (8,)
Dataset: BH_Mass, dtype: float32, shape: (8,)
Dataset: BH_Mdot, dtype: float32, shape: (8,)
Dataset: BH_MdotBondi, dtype: float32, shape: (8,)
Dataset: BH_MdotEddington, dtype: float32, shape: (8,)
Dataset: BH_Pressure, dtype: float32, shape: (8,)
Dataset: BH_Progs, dtype: uint32, shape: (8,)
Dataset: BH_U, dtype: float32, shape: (8,)
Dataset: Coordinates, dtype: float64, shape: (8, 3)
Dataset: Masses, dtype: float32, shape: (8,)
Dataset: ParticleIDs, dtype: uint64, shape: (8,)
Dataset: Potential, dtype: float32, shape: (8,)
Dataset: SubfindDMDensity, dtype:

In [106]:
bh_schema = T.StructType([
    T.StructField("BH_BPressure", T.FloatType(), True),
    T.StructField("BH_CumEgyInjection_QM", T.FloatType(), True),
    T.StructField("BH_CumEgyInjection_RM", T.FloatType(), True),
    T.StructField("BH_CumMassGrowth_QM", T.FloatType(), True),
    T.StructField("BH_CumMassGrowth_RM", T.FloatType(), True),
    T.StructField("BH_Density", T.FloatType(), True),
    T.StructField("BH_HostHaloMass", T.FloatType(), True),
    T.StructField("BH_Hsml", T.FloatType(), True),
    T.StructField("BH_Mass", T.FloatType(), True),
    T.StructField("BH_Mdot", T.FloatType(), True),
    T.StructField("BH_MdotBondi", T.FloatType(), True),
    T.StructField("BH_MdotEddington", T.FloatType(), True),
    T.StructField("BH_Pressure", T.FloatType(), True),
    T.StructField("BH_Progs", T.IntegerType(), True),  # uint32 → safe mapping to IntegerType
    T.StructField("BH_U", T.FloatType(), True),
    T.StructField("Coordinates", T.ArrayType(T.DoubleType()), True),
    T.StructField("Masses", T.FloatType(), True),
    T.StructField("ParticleIDs", T.LongType(), True),
    T.StructField("Potential", T.FloatType(), True),
    T.StructField("SubfindDMDensity", T.FloatType(), True),
    T.StructField("SubfindDensity", T.FloatType(), True),
    T.StructField("SubfindHsml", T.FloatType(), True),
    T.StructField("SubfindVelDisp", T.FloatType(), True),
    T.StructField("Velocities", T.ArrayType(T.FloatType()), True),
])

In [107]:
bh_group = h5f['PartType5']

In [108]:
%%time
# Convert datasets to dictionary of NumPy arrays: this causes some type errors. 
#Let's use `list`, not `np.array`
# Extract and convert each dataset to a list
bh_data = {
    key: bh_group[key][:].tolist()
    for key in bh_group.keys()
}

CPU times: user 1.71 ms, sys: 3.37 ms, total: 5.08 ms
Wall time: 4.54 ms


### EDA Code Blocks for `bh_data`

In [110]:
# Optional: Confirm structure
for key, value in bh_data.items():
    print(f"{key}: type={type(value)}, shape={len(value)}")

BH_BPressure: type=<class 'list'>, shape=8
BH_CumEgyInjection_QM: type=<class 'list'>, shape=8
BH_CumEgyInjection_RM: type=<class 'list'>, shape=8
BH_CumMassGrowth_QM: type=<class 'list'>, shape=8
BH_CumMassGrowth_RM: type=<class 'list'>, shape=8
BH_Density: type=<class 'list'>, shape=8
BH_HostHaloMass: type=<class 'list'>, shape=8
BH_Hsml: type=<class 'list'>, shape=8
BH_Mass: type=<class 'list'>, shape=8
BH_Mdot: type=<class 'list'>, shape=8
BH_MdotBondi: type=<class 'list'>, shape=8
BH_MdotEddington: type=<class 'list'>, shape=8
BH_Pressure: type=<class 'list'>, shape=8
BH_Progs: type=<class 'list'>, shape=8
BH_U: type=<class 'list'>, shape=8
Coordinates: type=<class 'list'>, shape=8
Masses: type=<class 'list'>, shape=8
ParticleIDs: type=<class 'list'>, shape=8
Potential: type=<class 'list'>, shape=8
SubfindDMDensity: type=<class 'list'>, shape=8
SubfindDensity: type=<class 'list'>, shape=8
SubfindHsml: type=<class 'list'>, shape=8
SubfindVelDisp: type=<class 'list'>, shape=8
Veloci

In [111]:
%%time
# Turn the dictionary into row-wise tuples
bh_data_tuples = list(zip(*bh_data.values()))

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 15 µs


In [112]:
%%time
bhdf = spark.createDataFrame(bh_data_tuples, schema=bh_schema)

CPU times: user 3.8 ms, sys: 0 ns, total: 3.8 ms
Wall time: 12.5 ms


In [113]:
bhdf.printSchema()

root
 |-- BH_BPressure: float (nullable = true)
 |-- BH_CumEgyInjection_QM: float (nullable = true)
 |-- BH_CumEgyInjection_RM: float (nullable = true)
 |-- BH_CumMassGrowth_QM: float (nullable = true)
 |-- BH_CumMassGrowth_RM: float (nullable = true)
 |-- BH_Density: float (nullable = true)
 |-- BH_HostHaloMass: float (nullable = true)
 |-- BH_Hsml: float (nullable = true)
 |-- BH_Mass: float (nullable = true)
 |-- BH_Mdot: float (nullable = true)
 |-- BH_MdotBondi: float (nullable = true)
 |-- BH_MdotEddington: float (nullable = true)
 |-- BH_Pressure: float (nullable = true)
 |-- BH_Progs: integer (nullable = true)
 |-- BH_U: float (nullable = true)
 |-- Coordinates: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- Masses: float (nullable = true)
 |-- ParticleIDs: long (nullable = true)
 |-- Potential: float (nullable = true)
 |-- SubfindDMDensity: float (nullable = true)
 |-- SubfindDensity: float (nullable = true)
 |-- SubfindHsml: float (nullable = tru

In [114]:
%%time
rows = bhdf.take(5)  # Returns list of Row objects
rows_dict = [row.asDict() for row in rows]
df = pd.DataFrame(rows_dict)

# Transpose and label
df_transposed = df.transpose()
df_transposed.columns = [f"Row_{i}" for i in range(len(rows))]
print(df_transposed)

                                                                   Row_0  \
BH_BPressure                                                    0.625054   
BH_CumEgyInjection_QM                                        560238592.0   
BH_CumEgyInjection_RM                                        412865024.0   
BH_CumMassGrowth_QM                                              0.24934   
BH_CumMassGrowth_RM                                             0.018375   
BH_Density                                                      0.000151   
BH_HostHaloMass                                             14599.547852   
BH_Hsml                                                         1.789572   
BH_Mass                                                         0.277315   
BH_Mdot                                                         0.001269   
BH_MdotBondi                                                    0.001269   
BH_MdotEddington                                                4.442489   
BH_Pressure 

### All-in-one Code Block for `bh_data`

In [115]:
hdfsheader + hdfsbhpath

'hdfs://spark00:54310/common/data/illustris/tng50/snapshot84/bh/'

In [116]:
bhnametail = '_bh.parquet.snappy'

In [117]:
outbhname = hdfsheader + hdfsbhpath + parsedname + bhnametail
print(outbhname)

hdfs://spark00:54310/common/data/illustris/tng50/snapshot84/bh/snap_084.0_bh.parquet.snappy


In [118]:
%%time
bhdf.write.option("compression", "snappy") \
    .mode("overwrite") \
    .save(outbhname)

CPU times: user 0 ns, sys: 7.38 ms, total: 7.38 ms
Wall time: 1.21 s


# Let's see HDFS files

In [125]:
!hdfs dfs -du -s -h /common/data/illustris/tng50/snapshot84/dm/*

857.4 M  2.5 G  /common/data/illustris/tng50/snapshot84/dm/snap_084.0_dm.parquet.snappy


In [126]:
!hdfs dfs -du -s -h /common/data/illustris/tng50/snapshot84/star/*

353.9 M  1.0 G  /common/data/illustris/tng50/snapshot84/star/snap_084.0_star.parquet.snappy


In [127]:
!hdfs dfs -du -s -h /common/data/illustris/tng50/snapshot84/gas/*

2.3 G  7.0 G  /common/data/illustris/tng50/snapshot84/gas/snap_084.0_gas.parquet.snappy


In [128]:
!hdfs dfs -du -s -h /common/data/illustris/tng50/snapshot84/bh/*

52.8 K  158.3 K  /common/data/illustris/tng50/snapshot84/bh/snap_084.0_bh.parquet.snappy
